## Leverage some drawing capabilities"

In [1]:
# import the module Turtle
import turtle as t

## Step 1: Data model setup

In [2]:
NAME = 0
POINTS = 1
POP = 2

state = ["COLORADO",               # label for the state
         [[-109, 37], [-109, 41],  # list of list that contains latitude and longtitude data
          [-102, 41], [-102, 37]], # each of this pair of coordinates represnt the corners of the plot
         5187582]                  # size of population in the state

## Step 2: Expand data
- let's add some information about cities nested in the focal state
- the cities will be stored as nested lists
- each city's location consists of a single point as a longitude and latitude pair

In [3]:
cities = []
cities.append(["DENVER",[-104.98, 39.74], 634265])
cities.append(["BOULDER",[-105.27, 40.02], 98889])
cities.append(["DURANGO",[-107.88,37.28], 17069])

## Step 3: Fix map size
- we will now render our GIS data as a map by first defining a map size
- the width and height can be anything that you want depending on your screen resolution

In [5]:
map_width = 400
map_height = 300

# this define the size of the polygons (Colarado state)
# all of the cities will fix within the space

## Step 4: Scaling the map
- in order to scale the map to the graphics canvas, we must first determine the bounding box of the largest layer, which is the state
- we'll set the map's bounding box to a global scale and reduce it to the size of the state. To do so:
- we'll loop through the longitude and latitude of each point and compare it with the current minimum and maximum x and y values
- if it is larger than the current maximum or smaller than the current minimum, we'll make this value the new maximum or minimum, respectively

In [9]:
# for the state

minx = 105
maxx = -112
miny = 40
maxy = -44

for x, y in state[POINTS]:
    if x < minx: minx = x    # if the x-coor are lesser than minx, then change it to minx
    elif x > maxx: maxx = x  # if the x-coor are larger than maxx, then change it to maxx
    if y < miny: miny = y
    elif y > maxy: maxy = y
        
# this step is to make sure the state polygons (corners) fall within the bounding box

### Scaling the map for the cities
- the second step to scaling is to calculate a ratio between the actual state and the tiny canvas that we will render it on
- this ratio is used for coordinate to pixel conversion. We get the size along the x and y axes of the state and then we divide the map width and height by these numbers to get our scaling ratio:

In [11]:
dist_x = maxx - minx   # distance between maxx and minx
dist_y = maxy - miny   # distance between maxy and miny

x_ratio = map_width / dist_x   # these ratios are used to convert coord points to pixels, so we can plot them
y_ratio = map_height / dist_y

## Step 5: Sample GIS Function
- be 'convert()' our sample GIS function
- we require convert() to transform a point in the map coordinates from one of our data layers to pixel coordinates using the previous calculations
- in order to to account for the unusual center origin of the turtle graphics canvas, we divide the map width and height in half and subtract it from the final conversion

In [12]:
# What we are doing here is passing pairs of longtitude & latitude points to 
# pairs of data express in terms of pixels (x & y).

def convert(point):
    lon = point[0]   
    lat = point[1]
    x = map_width - ((maxx - lon) * x_ratio)
    y = map_height - ((maxy - lat) * y_ratio)
   
    # Python turtle graphics start in the
    # middle of the screen
    # so we must offset the points so they are centered
    x = x - (map_width/2)
    y = y - (map_height/2)
    return [x,y]

## Step 6: Render our GIS as a thematic map
- the turtle module uses the concept of a cursor called a pen
- moving the cursor around the canvas is exactly the same as moving a pen around a piece of paper
- the cursor will draw a line when you move it―so, you'll notice that throughout the code, we use the t.up() and t.down() commands to pick the pen up when we want to move to a new location and put it down when we're ready to draw. We have some important steps in this section
- as the border of Colorado is a polygon, we must draw a line between the last point and first point to close the polygon. We can also leave out the closing step and just add a duplicate point to the Colorado dataset
- once we draw the state, we'll use the write() method to label the polygon

In [13]:
first_pixel = None
t.up()   # hold the pen up, so not drawing anything

for point in state[POINTS]:   # state[POINTS] = longtitude + latitude
    pixel = convert(point)    # convert the coor into pixels (x & y)
    if not first_pixel:
        first_pixel=pixel
    t.goto(pixel)   # go to the point
    t.down()        # draw the point


t.goto(first_pixel)
t.up()
t.goto([0,0])
t.write(state[NAME], align="center", font=("Arial",16,"bold"))

### Render our GIS as a thematic map for cities
- now, we'll render the cities as point locations and label them with their names and population;
- as the cities are a group of features in a list, we'll loop through them to render them. Instead of drawing lines by moving the pen around, we'll use the turtle dot() method to plot a small circle at the pixel coordinate returned by our SimpleGISconvert() function
- you'll notice that we must convert the population number to a string in order to use it in the turtle write() method. To do so, we use Python's built-in function, str()

In [14]:
for city in cities:
    pixel = convert(city[POINTS])
    t.up()
    t.goto(pixel)
    # Place a point for the city
    t.dot(10)
    # Label the city
    t.write(city[NAME] + ", Pop.: " + str(city[POP]), align="left")
    t.up()